In [0]:
from pyspark.sql.types import StructType
from pyspark.sql.functions import current_timestamp, from_utc_timestamp


In [0]:
path_schema = '/Volumes/ingestao_dados/tabnews/dadostabnews/Ano_2025/Mês_01/Dia_01/Minuto_01/Segundo_08/96a2b741-e363-4146-b147-e4e3b8df9b23.json'
sample_df = spark.read.option('multiline', 'true').json(path_schema)

In [0]:
path = '/Volumes/ingestao_dados/tabnews/dadostabnews/Ano_2025/Mês_01/Dia_01/Minuto_01/Segundo_08/*.json'
df = (spark.read.option('multiline', 'true')
                .schema(sample_df.schema)
                .json(path))
df_com_data_requisicao = df.withColumn('data_da_requisicao', from_utc_timestamp(current_timestamp(), "America/Sao_Paulo"))
df_com_data_requisicao.createOrReplaceTempView('Raw_TabNews_2025')

In [0]:
%sql

SELECT * 
FROM Raw_TabNews_2025
QUALIFY ROW_NUMBER() OVER (PARTITION BY id ORDER BY UPDATED_AT DESC) = 1

In [0]:
(df_com_data_requisicao.write.mode('overwrite')
                       .format('delta')
                       .option('OverWriteSchema', 'true')
                       .saveAsTable('databricks_lagodomago.trampar_lakehouse.Bronze_TabNews'))